QC Array
========

***Analysis step***

**Author:** *Jay Kim*

## Test sample contamination (bafRegress)
G. Jun, M. Flickinger, K. N. Hetrick, Kurt, J. M. Romm, K. F. Doheny, G. Abecasis, M. Boehnke,and H. M. Kang, _Detecting and Estimating Contamination of Human DNA Samples in Sequencing and Array-Based Genotype Data_, American journal of human genetics doi:10.1016/j.ajhg.2012.09.004 (volume 91 issue 5 pp.839 - 848)

### Read in bead pool manifest and fetch sample metadata from the database

In [4]:
library(dplyr, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)
library(glue)

GTC_BUCKET <- "pbc-iscan-qcarrays"

# Get the bead pool manifest data
bead_array_files <- reticulate::import("IlluminaBeadArrayFiles")
manifest <- bead_array_files$BeadPoolManifest(glue("s3://{GTC_BUCKET}/InfiniumQCArray-24v1-0_A4.bpm"))

# Fetch the sample metadata from the database
sample_info <- open_dataset("s3://pbc-qcarray-sample-info/", partitioning=c("JIRA"))
#sample_info %>% collect() %>% head()

### Get baf and genotype data for all samples in the project/batch:

In [5]:
# Get baf and genotype data for each sample in the sample list
get_baf_data <- function(Barcode, Position) {
    gtc <- bead_array_files$GenotypeCalls(glue("s3://{GTC_BUCKET}/{Barcode}/{Position}.gtc"))
    baf <- gtc$get_ballele_freqs()
    abgeno <- gtc$get_genotypes()
    tibble(baf=baf,abgeno=abgeno) %>% 
        mutate(abgeno=bead_array_files$code2genotype[abgeno+1])
}
baf_data <- sample_info %>%
    filter(JIRA == "JEWSC_20211118_Qcarray") %>%
    group_by(Sample_ID) %>%
    collect() %>%
    summarize(get_baf_data(Barcode,Position), .groups="keep")

### Aggregate MAFs across multiple projects/batches:

In [6]:
source("popmaf.R")

# Fetch the sample counts per project from the metadata in each sample_info file. First query all
# the available JIRA values that match the condition, then get the row count from the file metadata.
# This should be very fast.
batches <- sample_info %>%
    filter(JIRA %in% c("JEWSC_20211118_Qcarray")) %>%
    select(JIRA) %>% distinct(JIRA) %>%
    collect() %>%
    mutate(nsamples = num_rows(JIRA))

# Now aggregate all of the per batch MAFs, then combine the whole range requested
popmaf <- calc_popmaf(batches, GTC_BUCKET)

### Test each sample for contamination:

In [96]:
source("bafRegress.R")
baf_results <- baf_data %>%
    map(~ testsamplecontamination(.$baf,.$abgeno,popmaf$maf))

# Analysis

set the name of comparison prefix by pulling the parent directory name

In [ ]:
comparison = tail(strsplit(getwd(),"/")[[1]],n=1)

## Import gencalls

In [124]:
source("analysis-utils.R")
raw_data <- import.gencalls(sample_info, manifest)
summary(raw_data)
head(raw_data)

[1] 2
[1] 2
CPT0nnnnn_0001 CPT0nnnnn_0002 
             2              2 
--- raw_data ---
A genotypes object with 15949 sites x 2 samples
Allele encoding: native 
Intensity data: yes (raw) 
Sample metadata: yes ( 0 male / 2 female / 0 unknown )
Filters set: 0 sites / 0 samples 
Checksum: 2ebb6487683dbda81c6458d34d10d644 
Genotypes matrix:
                CPT0n CPT0n 
 2010-08-Y-1111     H     H 
 2010-08-Y-1221     H     H 
 2010-08-Y-1995     H     H 
 2010-08-Y-2045     H     H 
 2010-08-Y-3042     H     H 
 2010-08-Y-3189     H     H 
 2010-08-Y-3314     H     H 
 2010-08-Y-3348     H     H 
 2010-08-Y-3576     H     H 
  2010-08-Y-749     H     H 

Marker map:
 chr         marker cM pos A1 A2
   0 2010-08-Y-1111 NA   0  A  G
   0 2010-08-Y-1221 NA   0  A  G
   0 2010-08-Y-1995 NA   0  A  C
   0 2010-08-Y-2045 NA   0  A  G
   0 2010-08-Y-3042 NA   0  T  C
   0 2010-08-Y-3189 NA   0  A  C
   0 2010-08-Y-3314 NA   0  T  C
   0 2010-08-Y-3348 NA   0  T  C
   0 2010-08-Y-3576 NA   0  T

## QC .idat

In [ ]:
# ...

## Import .gds file

In [ ]:
# ...